# Import libraries

In [1]:
import eumdac
from datetime import datetime, timedelta
import shutil
import requests
import time
import fnmatch
import h5py
import os
import re
import numpy as np
import xarray as xr



from pathos.threading import ThreadPool as Pool

# Set paths

In [2]:
# OUTPUTDIR = '../../projects/4868_10/Datasets/MSG/raw/'
OUTPUTDIR = r'/DATAFOLDER/cluster_projects/ro/1149_10/SEVIRI_retrieval/Native_files'

# Set API credentials

### Tokens can be found here: https://api.eumetsat.int/api-key/

In [3]:
# Insert your personal key and secret into the single quotes
consumer_key = 'f63FWWWCcZ72ToRnT04xmT81QRUa' #HKV account
consumer_secret = '5i55hTcN2EumgTp4GACmklVRqJUa' # HKV account

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

datastore = eumdac.DataStore(token)
datatailor = eumdac.DataTailor(token)

try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as error:
    print(f"Unexpected error: {error}")

This token '246493c8-1a70-3b63-baed-8a158e72f162' expires 2025-01-14 14:45:12.006383


# Define collection and timespan

In [4]:
# Define collection
collection = 'EO:EUM:DAT:MSG:HRSEVIRI'

# Set sensing start and end time
start = datetime(2018, 6, 1, 0, 0)
end = datetime(2018, 6, 30, 23, 59)

# Get dataset info

In [5]:
# Select collection from datastore
# datastore = eumdac.DataStore(token)

try:    
    selected_collection = datastore.get_collection(collection)
    print(f"{selected_collection} - {selected_collection.title}")
except eumdac.datastore.DataStoreError as error:
    print(f"Error related to the data store: '{error.msg}'")
except eumdac.collection.CollectionError as error:
    print(f"Error related to the collection: '{error.msg}'")
except requests.exceptions.ConnectionError as error:
    print(f"Error related to the connection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

EO:EUM:DAT:MSG:HRSEVIRI - High Rate SEVIRI Level 1.5 Image Data - MSG - 0 degree


# Retrieve file names

In [6]:
# Retrieve datasets that match our filter
products = selected_collection.search(
    dtstart=start,
    dtend=end)
print(f'Found Datasets: {products.total_results} datasets for the given time range')


# for product in products:
#     try:
#         print(product)
#     except eumdac.collection.CollectionError as error:
#         print(f"Error related to the collection: '{error.msg}'")
#     except requests.exceptions.ConnectionError as error:
#         print(f"Error related to the connection: '{error.msg}'")
#     except requests.exceptions.RequestException as error:
#         print(f"Unexpected error: {error}")

Found Datasets: 2877 datasets for the given time range


# Get collection function

In [7]:
def get_collection(collection, start_time, end_time, credentials):
    
    token = eumdac.AccessToken(credentials)

    datatailor = eumdac.DataTailor(token)
    datastore = eumdac.DataStore(token)

    try:    
        selected_collection = datastore.get_collection(collection)
        print(f"{selected_collection} - {selected_collection.title}")
    except eumdac.datastore.DataStoreError as error:
        print(f"Error related to the data store: '{error.msg}'")
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except requests.exceptions.ConnectionError as error:
        print(f"Error related to the connection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

    # Retrieve datasets that match our filter
    products = selected_collection.search(
        dtstart=start,
        dtend=end)
    print(f'Found Datasets: {products.total_results} datasets for the given time range')

    # Create nested list of products for parallel pool
    nested_products = [[x] for x in products]
    list_of_dirs = [OUTPUTDIR] * len(nested_products)
    return nested_products, list_of_dirs 

# Define bounding box area

In [8]:
# Bounding box (in degrees)
min_lon = -3.7
max_lon = 1.35
min_lat = 4.5
max_lat = 11.3

# Define chain

In [9]:
chain = eumdac.tailor_models.Chain(
    product='HRSEVIRI',
    format='msgnative',
    filter={"bands": ["channel_1", "channel_2", "channel_3", "channel_4", "channel_5", "channel_6", "channel_7", "channel_8", "channel_9", "channel_10", "channel_11"]},
    #projection="mercator",
    roi={"NSWE": [max_lat, min_lat, min_lon, max_lon]},
    #resample_method = 'near',
    #resample_resolution = 0.1
)

In [10]:
def import_SEVIRI(file_path):
    file_path = file_path

    scn = Scene(reader="seviri_l1b_native", filenames=[file_path])
    #sc = Scene(filenames=[file_path], reader="fci_l1c_nc")
    #sc = Scene(filenames=[file_path], reader="satpy_cf_nc")
    
    #print(scn.all_dataset_names())

    scn.load(['IR_016', 'IR_039', 'IR_087', 'IR_097', 'IR_108', 'IR_120', 'IR_134', 'VIS006', 'VIS008', 'WV_062', 'WV_073'])
    return scn

# Linear processing

In [11]:
def download_api_products(products, output_dir):
    #    Create a list of SEVIRI bands'
    
    sleep_time = 10
    output_dir = output_dir

    for product in products:
        # if not os.path.exists(fr'{output_dir}\\{product}.hdf5'): 
        # if not os.path.exists(fr'{output_dir}\\{product}.subset.nat'):
        year = str(product).split('-')[5][0:4]
        month = str(product).split('-')[5][4:6]
        output_direc = os.path.join(output_dir, year, month)
        os.makedirs(output_direc, exist_ok = True)
        
        if not any(str(product).split('.')[0] in filename for filename in os.listdir(output_direc)):
            try:
                # start_c = time.time()
                customisation = datatailor.new_customisation(product, chain)
                #print(f"Customisation {customisation._id} started.")
            # except eumdac.datatailor.DataTailorError as error:
            #     print(f"Error related to the Data Tailor: '{error.msg}'")
            # except requests.exceptions.RequestException as error:
            #     print(f"Unexpected error: {error}")
    
                while True:
                    status = customisation.status
                    if "DONE" in status:
                        # stop_c = time.time()
                        # print(f'customization took {(stop_c - start_c)} sec')
                        # start_d = time.time()
                        #print(f"Customisation {customisation._id} is successfully completed.")
                        #print(f"Downloading the msgnative output of the customisation {customisation._id}")
                        zip_files = fnmatch.filter(customisation.outputs, '*')[0]
                        with customisation.stream_output(zip_files) as stream:
                            # Check if stream.name (the file path) already exists
                            if not os.path.exists(fr'{output_direc}\\{stream.name}'):
                                # Customize the file path to include the custom directory
                                custom_file_path = os.path.join(output_direc, os.path.basename(stream.name))
                                # If the file doesn't exist, open it for writing
                                with open(custom_file_path, mode='wb') as fdst:
                                    shutil.copyfileobj(stream, fdst)
    
                                print(f"File '{stream.name}' created and saved.")
                            else:
                                print(f"File '{stream.name}' already exists. Skipping creation.")
                        
                        # stop_d = time.time()
                        # print(f'downloading took {(stop_d - start_d)} sec')
                        print(f"Download finished for customisation {customisation._id}.")
    
                        # start_rpj = time.time()
                    
                        # The server only has 20 GB available for customizations, make sure to delete them after they have been dowloaded
                        customisation.delete()
    
                        break
                    elif status in ["ERROR", "FAILED", "DELETED", "KILLED", "INACTIVE"]:
                        print(f"Customisation {customisation._id} was unsuccessful. Comisausttion log is printed.\n")
                        print(customisation.logfile)
                        try:
                            customisation.delete()
                        except eumdac.datatailor.CustomisationError as error:
                            print("Customisation Error:", error)
                        except Exception as error:
                            print("Unexpected error:", error)
                        break
                    elif "QUEUED" in status:
                        print(f"Customisation {customisation._id} is queued.")
                    # elif "RUNNING" in status:
                    #     print(f"Customisation {customisation._id} is running.")
                    time.sleep(sleep_time)    
    

            except:# requests.HTTPError as exc:
                 # Clearing all customisations from the Data Tailor
                print('unexpected error occured')
                for customisation in datatailor.customisations: 
                    if customisation.status in ['INACTIVE']:
                        customisation.kill()
                        try:
                            customisation.delete()
                        except eumdac.datatailor.CustomisationError as error:
                            print("Customisation Error:", error)
                        except Exception as error:
                            print("Unexpected error:", error)
                        
                        print(f'Delete {customisation.status} customisation {customisation} from {customisation.creation_time} UTC.')
            
                    elif customisation.status in ["ERROR", "FAILED", "DELETED", "KILLED",]:
                        try:
                            customisation.delete()
                        except eumdac.datatailor.CustomisationError as error:
                            print("Customisation Error:", error)
                        except requests.exceptions.RequestException as error:
                            print("Unexpected error:", error)

                        print(f'Delete completed customisation {customisation} from {customisation.creation_time} UTC.')

               
                

            
                   
    return

# Parallel processing

In [12]:
def parallel_download_api_products(list_of_products, list_of_dirs, threads=3):
    # Set number of threads (cores) used for parallel run and map threads
    if threads is None:
        pool = Pool()
    else:
        pool = Pool(nodes=threads)
    # Run parallel function
    results = pool.map( download_api_products,
                        list_of_products, list_of_dirs)
    
    
    return results

In [13]:
# # Create nested list of products for parallel pool
# nested_products = [[x] for x in products]
# list_of_dirs = [OUTPUTDIR] * len(nested_products)

In [14]:
# # Parallel processing with timing
# start = time.time()
# parallel_download_api_products(nested_products, list_of_dirs)
# stop = time.time()
# print(f'Execution time (minutes): {(stop-start)/60}')

In [ ]:
start = datetime(2017, 1, 1, 0, 0)
end = datetime(2017, 1, 1, 23, 59)

for i in range(365):
    # Define collection
    collection = 'EO:EUM:DAT:MSG:HRSEVIRI'
    # Set sensing start and end time
    nested_products, list_of_dirs = get_collection(collection, start, end, credentials)
    parallel_download_api_products(nested_products, list_of_dirs)
    start += timedelta(days = 1)
    end += timedelta(days = 1)
    print(start)

EO:EUM:DAT:MSG:HRSEVIRI - High Rate SEVIRI Level 1.5 Image Data - MSG - 0 degree
Found Datasets: 96 datasets for the given time range
File 'MSG3-SEVI-MSG15-0100-NA-20170101064240.623000000Z-NA.subset.nat' created and saved.
Download finished for customisation ab87c02b.
File 'MSG3-SEVI-MSG15-0100-NA-20170101035740.178000000Z-NA.subset.nat' created and saved.
Download finished for customisation 3df37bce.
2017-01-02 00:00:00
EO:EUM:DAT:MSG:HRSEVIRI - High Rate SEVIRI Level 1.5 Image Data - MSG - 0 degree
Found Datasets: 96 datasets for the given time range
2017-01-03 00:00:00
EO:EUM:DAT:MSG:HRSEVIRI - High Rate SEVIRI Level 1.5 Image Data - MSG - 0 degree
Found Datasets: 96 datasets for the given time range
Customisation 35693192 is queued.
File 'MSG3-SEVI-MSG15-0100-NA-20170103105740.087000000Z-NA.subset.nat' created and saved.
Download finished for customisation 35693192.
2017-01-04 00:00:00
EO:EUM:DAT:MSG:HRSEVIRI - High Rate SEVIRI Level 1.5 Image Data - MSG - 0 degree
Found Datasets:

# Clearing customization workspace
If you have fully used your workspace (20 GB) for the customisations you can run this cell to clear them 

In [ ]:
 # Clearing all customisations from the Data Tailor

for customisation in datatailor.customisations:
    if customisation.status in ['QUEUED', 'INACTIVE', 'RUNNING']:
        customisation.kill()
        print(f'Delete {customisation.status} customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except Exception as error:
            print("Unexpected error:", error)
    else:
        print(f'Delete completed customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except requests.exceptions.RequestException as error:
            print("Unexpected error:", error)